In [1]:
import csv
import os
import re

In [2]:
def read_csv(results_path="//store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/output/new_results/edits/batch"):
    dirs = [os.path.join(results_path, d) for d in os.listdir(results_path)]
    files = [os.path.join(d, "edits.csv") for d in dirs if "edits.csv" in os.listdir(d)]

    dict = {}

    for filename in files:
        name = re.split(r"\\", filename)[1]
        with open(filename, "r") as f:
            reader = list(csv.DictReader(f))
            for row in reader:
                dict[row["query_index"]] = (dict[row["query_index"]] if row["query_index"] in dict else {}) | {
                    name: {
                        "query_edit": row["query_edit"],
                        "distractor_edit": row["distractor_edit"]
                    }
                }

    return dict

In [3]:
dict = read_csv()

In [4]:
query_matches = 0
distractor_matches = 0
identical_matches = 0
partial_matches = 0

matches = {}

for query_index, d in dict.items():
    query_edits = [e["query_edit"] for e in list(d.values())]
    distractor_edits = [e["distractor_edit"] for e in list(d.values())]
    query_identical = query_edits.count(query_edits[0]) == len(query_edits)
    distractor_identical = distractor_edits.count(distractor_edits[0]) == len(distractor_edits)
    if query_identical:
        query_matches += 1
    if distractor_identical:
        distractor_matches += 1
    if query_identical and distractor_identical:
        identical_matches += 1        
        partial_matches += 1
        matches[query_index] = "identical"
    elif query_identical or distractor_identical:
        partial_matches += 1
        matches[query_index] = "partial"
    else:
        matches[query_index] = "none"

print(f"query match: {query_matches / len(dict)}")
print(f"distractor match: {distractor_matches / len(dict)}")
print(f"identical match: {identical_matches / len(dict)}")
print(f"partial match: {partial_matches / len(dict)}")

query match: 0.7129902516436182
distractor match: 0.5871684425300385
identical match: 0.4676944003627295
partial match: 0.8324642938109272


In [5]:
path = "//store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/output/new_results/edits/matches.csv"
with open(path, "w", newline='') as f:
    writer = csv.DictWriter(f, fieldnames=["query_index", "match"])
    writer.writeheader()
    for query_index, m in matches.items():
        writer.writerow({
            "query_index": query_index,
            "match": m,
        })